In [ ]:
from typing import Optional, Dict, Any
from enum import Enum
import modelcontextprotocol as mcp
from pydantic import BaseModel, Field

# Custom error types
class MCPErrorType(Enum):
    VALIDATION = "validation"
    RUNTIME = "runtime"
    SYSTEM = "system"
    USER = "user"
    
class MCPError(Exception):
    """Base class for MCP errors."""
    def __init__(self, message: str, error_type: MCPErrorType, details: Optional[Dict[str, Any]] = None):
        self.message = message
        self.error_type = error_type
        self.details = details or {}
        super().__init__(self.message)
        
class ValidationError(MCPError):
    """Raised when input validation fails."""
    def __init__(self, message: str, details: Optional[Dict[str, Any]] = None):
        super().__init__(message, MCPErrorType.VALIDATION, details)
        
class RuntimeError(MCPError):
    """Raised when an error occurs during tool execution."""
    def __init__(self, message: str, details: Optional[Dict[str, Any]] = None):
        super().__init__(message, MCPErrorType.RUNTIME, details)
        
class SystemError(MCPError):
    """Raised when a system-level error occurs."""
    def __init__(self, message: str, details: Optional[Dict[str, Any]] = None):
        super().__init__(message, MCPErrorType.SYSTEM, details)
        
class UserError(MCPError):
    """Raised when user input or action causes an error."""
    def __init__(self, message: str, details: Optional[Dict[str, Any]] = None):
        super().__init__(message, MCPErrorType.USER, details)
        
# Input/Output models
class ErrorResponse(BaseModel):
    error_type: MCPErrorType
    message: str
    details: Optional[Dict[str, Any]] = None
    
class CalculatorInput(BaseModel):
    operation: str = Field(..., description="Mathematical operation to perform")
    x: float = Field(..., description="First number")
    y: float = Field(..., description="Second number")
    
class CalculatorOutput(BaseModel):
    result: float
    operation: str


In [ ]:
class Calculator:
    """A calculator tool that demonstrates error handling."""
    
    VALID_OPERATIONS = {
        '+': lambda x, y: x + y,
        '-': lambda x, y: x - y,
        '*': lambda x, y: x * y,
        '/': lambda x, y: x / y,
        '**': lambda x, y: x ** y
    }
    
    def validate_input(self, input_data: CalculatorInput):
        """Validate calculator input."""
        if input_data.operation not in self.VALID_OPERATIONS:
            raise ValidationError(
                f"Invalid operation: {input_data.operation}",
                details={
                    "valid_operations": list(self.VALID_OPERATIONS.keys()),
                    "provided": input_data.operation
                }
            )
            
        if input_data.operation == '/' and input_data.y == 0:
            raise UserError(
                "Division by zero",
                details={
                    "x": input_data.x,
                    "y": input_data.y,
                    "operation": input_data.operation
                }
            )
            
        if input_data.operation == '**' and input_data.y > 100:
            raise ValidationError(
                "Exponent too large",
                details={
                    "max_exponent": 100,
                    "provided": input_data.y
                }
            )
    
    async def calculate(self, input_data: CalculatorInput) -> CalculatorOutput:
        """Perform calculation with error handling."""
        try:
            # Validate input
            self.validate_input(input_data)
            
            # Perform calculation
            operation_func = self.VALID_OPERATIONS[input_data.operation]
            result = operation_func(input_data.x, input_data.y)
            
            # Check for overflow
            if abs(result) > 1e308:  # Max float value
                raise RuntimeError(
                    "Result too large",
                    details={
                        "result": "overflow",
                        "operation": input_data.operation
                    }
                )
                
            return CalculatorOutput(
                result=result,
                operation=input_data.operation
            )
            
        except MCPError:
            # Re-raise MCP errors
            raise
        except Exception as e:
            # Convert other exceptions to MCPError
            raise SystemError(
                f"Unexpected error: {str(e)}",
                details={
                    "error_type": type(e).__name__,
                    "operation": input_data.operation
                }
            )

# Create calculator tool
calculator = Calculator()

# Create MCP server
server = mcp.Server()
server.add_tool("calculator", calculator.calculate, CalculatorInput, CalculatorOutput)


In [ ]:
import asyncio

async def test_calculator():
    # Test valid operations
    print("Testing valid operations:")
    try:
        result = await calculator.calculate(CalculatorInput(operation="+", x=5, y=3))
        print(f"5 + 3 = {result.result}")
    except MCPError as e:
        print(f"Error: {e.message}")
        
    # Test invalid operation
    print("\nTesting invalid operation:")
    try:
        result = await calculator.calculate(CalculatorInput(operation="%", x=10, y=2))
        print(result)
    except MCPError as e:
        print(f"Error type: {e.error_type}")
        print(f"Message: {e.message}")
        print(f"Details: {e.details}")
        
    # Test division by zero
    print("\nTesting division by zero:")
    try:
        result = await calculator.calculate(CalculatorInput(operation="/", x=1, y=0))
        print(result)
    except MCPError as e:
        print(f"Error type: {e.error_type}")
        print(f"Message: {e.message}")
        print(f"Details: {e.details}")
        
    # Test exponent too large
    print("\nTesting large exponent:")
    try:
        result = await calculator.calculate(CalculatorInput(operation="**", x=2, y=1000))
        print(result)
    except MCPError as e:
        print(f"Error type: {e.error_type}")
        print(f"Message: {e.message}")
        print(f"Details: {e.details}")

# Run the tests
await test_calculator()
